In [19]:
# Machine Learning toolkit
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# Python SQL toolkit and Object Relational Mapper
import sqlite3

In [2]:
con = sqlite3.connect("./beer_data.sqlite")
c = con.cursor()

In [3]:
beers_df = pd.read_sql_query("SELECT * FROM taste_profiles", con)
beer_stats = pd.read_sql_query("SELECT * FROM beer_stats", con)
reviews_df = pd.read_sql_query("SELECT * FROM reviews", con)

In [4]:
# Insert Description column
reviews_df.insert(3, 'Description', pd.read_sql_query("SELECT Description FROM beer_labels", con).values)

# Format Description column
remove_notes = [val.replace('Notes:', '') for val in reviews_df['Description'].values]
reviews_df['Description'] = remove_notes

reviews_df['Description'] = reviews_df['Description'].map(
    lambda desc: 'No description available.' if desc == '' else desc
)

In [5]:
# Round review scores to 2 decimal places
for col in reviews_df.columns[4:9]:
    reviews_df[col] = reviews_df[col].map("{:.2f}".format)
reviews_df

,Full_Beer_Name,Style,Brewery,Description,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
0,Alaskan Brewing Co. Alaskan Amber,Altbier,Alaskan Brewing Co.,"Richly malty and long on the palate, with just...",3.50,3.64,3.56,3.64,3.85,497
1,Long Trail Brewing Co. Double Bag,Altbier,Long Trail Brewing Co.,"This malty, full-bodied double alt is also kno...",3.80,3.85,3.90,4.02,4.03,481
2,Long Trail Brewing Co. Long Trail Ale,Altbier,Long Trail Brewing Co.,Long Trail Ale is a full-bodied amber ale mode...,3.41,3.67,3.60,3.63,3.83,377
3,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Altbier,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,No description available.,4.15,4.03,4.15,4.21,4.01,368
4,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Altbier,Ninkasi Brewing Company,Called 'Dark Double Alt' on the label.Seize th...,3.62,3.97,3.73,3.77,3.82,96
...,...,...,...,...,...,...,...,...,...,...
3192,Cisco Brewers Inc. Winter Shredder,Winter Warmer,Cisco Brewers Inc.,No description available.,4.12,3.88,3.88,3.75,3.73,4
3193,RJ Rockers Brewing Company First Snow Ale,Winter Warmer,RJ Rockers Brewing Company,This hearty American pale ale contains a rich ...,3.95,3.75,3.76,3.79,3.73,50
3194,Natty Greene's Pub & Brewing Co. Red Nose Wint...,Winter Warmer,Natty Greene's Pub & Brewing Co.,No description available.,3.58,3.71,3.60,3.67,3.71,26
3195,Fish Brewing Company / Fishbowl Brewpub Fish T...,Winter Warmer,Fish Brewing Company / Fishbowl Brewpub,No description available.,3.90,3.89,3.86,3.90,3.88,87


In [6]:
# Check beer data
beers_df.head()

,Full_Beer_Name,Style,Brewery,ABV,Min_IBU,Max_IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
0,Alaskan Brewing Co. Alaskan Amber,Altbier,Alaskan Brewing Co.,5.3,25,50,13,32,9,47,74,33,0,33,57,8,111
1,Long Trail Brewing Co. Double Bag,Altbier,Long Trail Brewing Co.,7.2,25,50,12,57,18,33,55,16,0,24,35,12,84
2,Long Trail Brewing Co. Long Trail Ale,Altbier,Long Trail Brewing Co.,5.0,25,50,14,37,6,42,43,11,0,10,54,4,62
3,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Altbier,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,8.5,25,50,13,55,31,47,101,18,1,49,40,16,119
4,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Altbier,Ninkasi Brewing Company,7.2,25,50,25,51,26,44,45,9,1,11,51,20,95


In [7]:
# Scale numerical values
scaler = MinMaxScaler()

def scale_col_by_row(df, cols):
    # Scale values by row
    scaled_cols = pd.DataFrame(scaler.fit_transform(df[cols].T).T, columns=cols)
    df[cols] = scaled_cols
    return df

def scale_col_by_col(df, cols):
    # Scale values by column
    scaled_cols = pd.DataFrame(scaler.fit_transform(df[cols]), columns=cols)
    df[cols] = scaled_cols
    return df

# Scale values in tasting profile features (across rows)
beers_df = scale_col_by_row(beers_df, beers_df.columns[6:])

# Scale values in tasting profile features (across columns)
beers_df = scale_col_by_col(beers_df, beers_df.columns[6:])

# Scale values in chemical features (across columns)
beers_df = scale_col_by_col(beers_df, beers_df.columns[3:6])

# Peek at re-scaled data
beers_df.head()

,Full_Beer_Name,Style,Brewery,ABV,Min_IBU,Max_IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
0,Alaskan Brewing Co. Alaskan Amber,Altbier,Alaskan Brewing Co.,0.092174,0.384615,0.5,0.117117,0.288288,0.081081,0.423423,0.666667,0.297297,0.0,0.297297,0.513514,0.072072,1.0
1,Long Trail Brewing Co. Double Bag,Altbier,Long Trail Brewing Co.,0.125217,0.384615,0.5,0.142857,0.678571,0.214286,0.392857,0.654762,0.190476,0.0,0.285714,0.416667,0.142857,1.0
2,Long Trail Brewing Co. Long Trail Ale,Altbier,Long Trail Brewing Co.,0.086957,0.384615,0.5,0.225806,0.596774,0.096774,0.677419,0.693548,0.177419,0.0,0.161290,0.870968,0.064516,1.0
3,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Altbier,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,0.147826,0.384615,0.5,0.101695,0.457627,0.254237,0.389831,0.847458,0.144068,0.0,0.406780,0.330508,0.127119,1.0
4,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Altbier,Ninkasi Brewing Company,0.125217,0.384615,0.5,0.255319,0.531915,0.265957,0.457447,0.468085,0.085106,0.0,0.106383,0.531915,0.202128,1.0


In [8]:
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(beers_df['Style'].values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['Style'])
encode_df.head()

C:\Users\Jack\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Style_Altbier,Style_Barleywine,Style_Bitter,Style_Blonde Ale,Style_Bock,Style_Brown Ale,Style_Chile Beer,Style_Cream Ale,Style_Dubbel,Style_Farmhouse Ale,...,Style_Scotch Ale / Wee Heavy,Style_Scottish Ale,Style_Smoked Beer,Style_Sour,Style_Stout,Style_Strong Ale,Style_Tripel,Style_Wheat Beer,Style_Wild Ale,Style_Winter Warmer
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Merge the two DataFrames together and drop the Style column
encoded_styles_df = beers_df.merge(encode_df,left_index=True,right_index=True).drop("Style",1)
encoded_styles_df

C:\Users\Jack\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Full_Beer_Name,Brewery,ABV,Min_IBU,Max_IBU,Astringency,Body,Alcohol,Bitter,Sweet,...,Style_Scotch Ale / Wee Heavy,Style_Scottish Ale,Style_Smoked Beer,Style_Sour,Style_Stout,Style_Strong Ale,Style_Tripel,Style_Wheat Beer,Style_Wild Ale,Style_Winter Warmer
0,Alaskan Brewing Co. Alaskan Amber,Alaskan Brewing Co.,0.092174,0.384615,0.5,0.117117,0.288288,0.081081,0.423423,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Long Trail Brewing Co. Double Bag,Long Trail Brewing Co.,0.125217,0.384615,0.5,0.142857,0.678571,0.214286,0.392857,0.654762,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Long Trail Brewing Co. Long Trail Ale,Long Trail Brewing Co.,0.086957,0.384615,0.5,0.225806,0.596774,0.096774,0.677419,0.693548,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,0.147826,0.384615,0.5,0.101695,0.457627,0.254237,0.389831,0.847458,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Ninkasi Brewing Company,0.125217,0.384615,0.5,0.255319,0.531915,0.265957,0.457447,0.468085,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,Cisco Brewers Inc. Winter Shredder,Cisco Brewers Inc.,0.153043,0.538462,0.5,0.202703,0.500000,0.324324,0.472973,0.621622,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3193,RJ Rockers Brewing Company First Snow Ale,RJ Rockers Brewing Company,0.104348,0.538462,0.5,0.107143,0.221429,0.164286,0.114286,0.385714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3194,Natty Greene's Pub & Brewing Co. Red Nose Wint...,Natty Greene's Pub & Brewing Co.,0.118261,0.538462,0.5,0.083333,0.458333,0.250000,0.197917,0.541667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3195,Fish Brewing Company / Fishbowl Brewpub Fish T...,Fish Brewing Company / Fishbowl Brewpub,0.130435,0.538462,0.5,0.100000,0.327273,0.454545,0.636364,0.654545,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = encoded_styles_df[encoded_styles_df.columns[16:]].values
X = encoded_styles_df[encoded_styles_df.columns[2:16]].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=78)

In [11]:
# Define the checkpoint path and filenames
os.makedirs("./ML_Weight_Checkpoints/", exist_ok=True)
checkpoint_path = "./ML_Weight_Checkpoints/seg2_model.h5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=10)

In [12]:
# Define the model
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 15
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=38, activation="softmax"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                225       
                                                                 
 dense_1 (Dense)             (None, 12)                192       
                                                                 
 dense_2 (Dense)             (None, 8)                 104       
                                                                 
 dense_3 (Dense)             (None, 38)                342       
                                                                 
Total params: 863
Trainable params: 863
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=200, callbacks=[cp_callback])

Epoch 1/200
75/75 [==============================] - 1s 3ms/step - loss: 3.5885 - accuracy: 0.0492
Epoch 2/200
75/75 [==============================] - 0s 3ms/step - loss: 3.3852 - accuracy: 0.0884
Epoch 3/200
75/75 [==============================] - 0s 3ms/step - loss: 3.1655 - accuracy: 0.1777
Epoch 4/200
75/75 [==============================] - 0s 3ms/step - loss: 3.0626 - accuracy: 0.2040
Epoch 5/200
75/75 [==============================] - 0s 3ms/step - loss: 2.9675 - accuracy: 0.2094
Epoch 6/200
75/75 [==============================] - 0s 3ms/step - loss: 2.8743 - accuracy: 0.2165
Epoch 7/200
75/75 [==============================] - 0s 2ms/step - loss: 2.8025 - accuracy: 0.2324
Epoch 8/200
75/75 [==============================] - 0s 3ms/step - loss: 2.7355 - accuracy: 0.2416
Epoch 9/200
75/75 [==============================] - 0s 3ms/step - loss: 2.6756 - accuracy: 0.2482
Epoch 10/200
74/75 [============================>.] - ETA: 0s - loss: 2.6204 - accuracy: 0.2513
Epoch 10: sav

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

25/25 - 0s - loss: 1.1600 - accuracy: 0.6325 - 241ms/epoch - 10ms/step
Loss: 1.159985065460205, Accuracy: 0.6324999928474426


In [16]:
# Test model with data from a random beer, simulating user's preference data
user_input = beers_df.sample().iloc[0][3:].values
user_input = user_input.reshape(1,-1)
user_input

array([[0.1426086956521739, 0.23076923076923078, 0.3,
        0.13333333333333333, 0.4222222222222222, 0.2777777777777778,
        0.14444444444444446, 0.9999999999999998, 0.28888888888888886,
        0.0, 0.8111111111111109, 0.16666666666666666,
        0.31111111111111106, 0.8111111111111109]], dtype=object)

In [21]:
# Output beer style based on taste profile inputs
index = nn.predict(np.asarray(user_input).astype(np.float64)).argmax()
predicted_style = encode_df.columns[index].split('_', 1)[1]
print(f"Predicted Beer Style: {predicted_style}")

1/1 [==============================] - 0s 37ms/step
Predicted Beer Style: Dubbel


In [75]:
# Function to find top 5 similar beers with or without same style
def similar_beers(beers_df, user_input, style, same_style):
    if same_style:
        # Locate beers of same style
        sim_beers_df = beers_df.loc[beers_df['Style'] == style].reset_index(drop=True)
    else:
        # Locate other similar beers
        sim_beers_df = beers_df.loc[beers_df["Style"] != style].reset_index(drop=True)

    # Get numeric data for similar beers
    sim_beers_data = sim_beers_df.iloc[:, 3:]

    # Find nearest neighbors
    search = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(sim_beers_data)
    _, queried_indices = search.kneighbors(user_input)

    # Top 5 recommendations
    recommends_df = sim_beers_df.loc[queried_indices[0][1:]]
    recommends_df = recommends_df[['Full_Beer_Name', 'Brewery', 'Style']]
    recommends_df = pd.merge(recommends_df, reviews_df, how='inner', on=['Full_Beer_Name', 'Brewery', 'Style'])
    recommends_df = recommends_df.sort_values(by=['review_overall'], ascending=False)
    return recommends_df

In [76]:
# Top 5 similar beers of same style (by overall review score)
top_5_same_style = similar_beers(beers_df, user_input, predicted_style, same_style=True)
top_5_same_style

C:\Users\Jack\anaconda3\envs\mlenv\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


,Full_Beer_Name,Brewery,Style,Description,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
0,Brouwerij St. Bernardus NV St. Bernardus Pater 6,Brouwerij St. Bernardus NV,Dubbel,No description available.,4.03,4.16,4.03,4.12,4.17,556
2,Brouwerij Westmalle Westmalle Trappist Dubbel,Brouwerij Westmalle,Dubbel,Westmalle Dubbel (7%) is a dark red-brown beer...,4.08,4.18,4.06,4.15,4.14,1026
1,The Lost Abbey Lost & Found Abbey Ale,The Lost Abbey,Dubbel,Modeled after the great Trappist and Monastic ...,3.98,3.89,3.86,3.98,3.92,284
3,Brouwerij Sterkens N.V. St. Sebastiaan Dark,Brouwerij Sterkens N.V.,Dubbel,No description available.,3.85,3.95,3.76,3.90,3.85,110
4,New Belgium Brewing Abbey Belgian Style Ale,New Belgium Brewing,Dubbel,Our award-winning Abbey stays true to traditio...,4.02,4.06,3.93,4.02,3.83,640


In [77]:
# Top 5 similar beers of other styles (by overall review score)
top_5_diff_style = similar_beers(beers_df, user_input, predicted_style, same_style=False)
top_5_diff_style

C:\Users\Jack\anaconda3\envs\mlenv\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


,Full_Beer_Name,Brewery,Style,Description,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
3,Bierbrouwerij De Koningshoeven B.V. La Trappe ...,Bierbrouwerij De Koningshoeven B.V.,Quadrupel (Quad),Quadrupel is La Trappe’s heaviest ale with a s...,4.07,4.08,4.05,4.11,3.96,725
2,Brouwerij Van Steenberge N.V. Piraat 9,Brouwerij Van Steenberge N.V.,Strong Ale,"Belgian Strong Pale Ale, re-fermented with Bor...",3.67,4.00,3.83,4.33,3.92,3
4,Les Trois Mousquetaires Weizenbock Grande Cuvée,Les Trois Mousquetaires,Bock,"A dark wheat beer, hearty and top-fermented, i...",3.83,3.96,4.04,3.83,3.88,23
0,Portsmouth Brewery Portsmouth Weizenbock,Portsmouth Brewery,Bock,A dark wheat bock beer. The addition of NH map...,3.92,4.05,3.88,3.92,3.83,20
1,Brouwerij Bosteels Pauwel Kwak,Brouwerij Bosteels,Strong Ale,Belgian Specialty Ale\t,3.72,3.85,3.74,3.74,3.70,580
